In [1]:
import os
import glob
import pandas as pd
from datetime import datetime
import time

In [4]:
variable_path_base = r'../data/variables'
variable_files_pattern = f'climatica_principal_res.zip'

In [5]:
variable_path_base = variable_path_base.split('/')
variable_files = glob.glob(os.path.join(*variable_path_base, variable_files_pattern))
print(variable_files)

['..\\data\\variables\\climatica_principal_res.zip']


In [6]:
#Se debe usar uno de los dos segmentos
#Segmento para quitar la hora y minutos parte
#Toma como base los registros de variables ya separados
dtypes = {
    'CodigoEstacion':'category','Etiqueta':'category'#,'Fecha': 'string'
}
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')

In [13]:
#Segmento para ajustar diferentes formatos de fecha
dtypes = {
    'CodigoEstacion':'category',
    'IdParametro':'category',
    'Etiqueta':'category',
    'DescripcionSerie':'category',
    'Frecuencia':'category',
    'Grado':'category',
    'Calificador':'category',
    'NivelAprobacion':'category',
    'Fecha': 'string',
}

usecols = {
    'CodigoEstacion',
    'IdParametro',
    'Etiqueta',
    'DescripcionSerie',
    'Frecuencia',
    'Fecha',
    'Valor',
    'Grado',
    'Calificador',
    'NivelAprobacion'
}

In [14]:
#https://stackoverflow.com/a/50129940/1828356
def autoconvert_datetime(value):
    formats = ['%Y-%m-%d %H:%M', '%d/%m/%Y %H:%M']
    result_format = '%Y-%m-%d %H:%M'
    for dt_format in formats:
        try:
            #print(dt_format, value)
            dt_obj = datetime.strptime(value, dt_format)
            return dt_obj.strftime(result_format)
        except Exception as e:
            #print(e)
            pass
    return value

In [15]:
%%time
start_time = time.time()
for f in variable_files:
    print(f'Archivo: {f}')
    print(f'Leyendo h:{time.asctime()}...')
    
    #Segmento para quitar hora
    #df = pd.read_csv(f, dtype = dtypes, parse_dates = ['Fecha'], date_parser = dateparse)
    
    #Segmento para ajustar formatos
    df = pd.read_csv(f, usecols = usecols, dtype = dtypes)
    
    print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

    etiqueta = df.iat[0,1]
    print(f'Etiqueta: {etiqueta}')
    
    print(df.head())
    #df.tail()

    #Segmento para ajustar formatos
    df.Fecha = df.Fecha.apply(autoconvert_datetime)
    
    #Verificar nombre del archivo de salida
    #p = os.path.join(*variable_path_base, f'var-dia-{etiqueta}-raw.csv')
    p = os.path.join(*variable_path_base, f'climatica_principal_res_fecha.csv')
    
    
    print(f'Guardando {p} h:{time.asctime()}...')
    
    df.to_csv(p, date_format = '%Y-%m-%d',  index = False)
    print(f'**** h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

Archivo: ..\data\variables\climatica_principal_res.zip
Leyendo h:Sun Jun 28 11:50:33 2020...
**** r:(1240194, 10) h:Sun Jun 28 11:50:40 2020 t:0.10345001220703125 ****
Etiqueta: TEMPERATURA
  CodigoEstacion  IdParametro Etiqueta           DescripcionSerie Frecuencia  \
0       12045010  TEMPERATURA  TMN_CON  Temperatura mínima diaria     Diaria   
1       12045010  TEMPERATURA  TMN_CON  Temperatura mínima diaria     Diaria   
2       12045010  TEMPERATURA  TMN_CON  Temperatura mínima diaria     Diaria   
3       12045010  TEMPERATURA  TMN_CON  Temperatura mínima diaria     Diaria   
4       12045010  TEMPERATURA  TMN_CON  Temperatura mínima diaria     Diaria   

              Fecha  Valor Grado Calificador NivelAprobacion  
0  1991-01-01 00:00   20.8    50         NaN             900  
1  1991-01-02 00:00   24.2    50         NaN             900  
2  1991-01-03 00:00   22.2    50         NaN             900  
3  1991-01-04 00:00   21.2    50         NaN             900  
4  1991-01-05 